In [1]:
!pip install datasets 
!pip install langdetect 
!pip install transformers
!pip install torch
!pip install tiktoken
!pip install sentencepiece

In [2]:
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import pandas as pd
from datasets import load_dataset
from langdetect import detect

In [4]:
# an empty list to store DataFrames for each language
dataframes = []

# Define the list of language configurations
# languages = ['de', 'es', 'fr', 'ru', 'tu']
languages = ['de', 'es']

# Load the dataset for each language configuration
for lang in languages:
    print(f"Loading MLSUM dataset for language: {lang}")
    dataset = load_dataset('mlsum', lang, trust_remote_code=True)

    # Convert to pandas DataFrame for easier manipulation
    df = pd.DataFrame(dataset['train'])

    # Add a column for language
    df['language'] = lang

    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into one
data = pd.concat(dataframes, ignore_index=True)

# Filter to keep relevant columns
data = data.drop('url', axis=1)

# Display the filtered DataFrame
print("Filtered Data with Language Detection:")
print(data.head())

Loading MLSUM dataset for language: de
Loading MLSUM dataset for language: es
Filtered Data with Language Detection:
                                                text  \
0  Transport im Viehwaggon, Fleischgeruch in der ...   
1  Marmorner Zebrastreifen, pompöse Gebäude: Sind...   
2  Wenn an diesem Montag die Landesvorsitzenden d...   
3  Das Portrait von 1791 zeigt Haitis Nationalhel...   
4  Neue Köpfe und alte Bekannte: Die neue Regieru...   

                                             summary    topic  \
0  Transport im Viehwaggon, Fleischgeruch in der ...  politik   
1  Marmorner Zebrastreifen, pompöse Gebäude: Sind...  politik   
2  Oskar Lafontaine gibt den Parteivorsitz der Li...  politik   
3  Die Wurzeln des Elends liegen in der Vergangen...  politik   
4  Schwarz-Gelb ist noch nicht jene Traumkoalitio...  politik   

                                               title        date language  
0  So war Auschwitz: Erinnerungen einer Holocaust...  00/01/2010       de  
1  

In [5]:
# save the DataFrame to a CSV file
# filtered_df.to_csv('mlsum_articles.csv', index=False)
# print("Data Saved to CSV.")

In [6]:
import torch
print(torch.__version__)

2.5.1+cu124


In [7]:
from transformers import TFMBartForConditionalGeneration, MBart50Tokenizer, TFBartForConditionalGeneration, BartTokenizer
from datasets import Dataset

2024-11-12 22:19:14.954331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-12 22:19:16.102118: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-11-12 22:19:16.102267: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [8]:
# Load the translation model (mBART)
translation_model_name = "facebook/mbart-large-50-many-to-one-mmt"
translation_model = TFMBartForConditionalGeneration.from_pretrained(translation_model_name)
translation_tokenizer = MBart50Tokenizer.from_pretrained(translation_model_name, src_lang="ar_AR", tgt_lang="en_XX")

# translation_tokenizer = MBart50TokenizerFast.from_pretrained(translation_model_name)

# Load the summarization model (BART or T5)
summarization_model_name = "facebook/bart-large-cnn"  # Change to "t5-base" for T5
summarization_model = TFBartForConditionalGeneration.from_pretrained(summarization_model_name)
summarization_tokenizer = BartTokenizer.from_pretrained(summarization_model_name)

2024-11-12 22:19:17.515133: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-11-12 22:19:17.515181: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-11-12 22:19:17.515207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nlp-proj): /proc/driver/nvidia/version does not exist
2024-11-12 22:19:17.515503: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropr

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
def translate_text(text, source_lang="fr_XX"):

    if not isinstance(text, str):
      print("text is not a string")
      text = str(text)

    # Tokenize text for translation
    translated = translation_tokenizer(text, return_tensors="tf", padding=True, truncation=True, max_length=1024)
    # translated = {k: v.to("cuda") for k, v in translated.items()}
    # translated = translated.to("cuda")  # Ensure tensors are on GPU

    # Translate text
    translated_tokens = translation_model.generate(**translated)
    translated_text = translation_tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_text

In [18]:
def fine_tuned_summarize_text(text, model, tokenizer, max_length=512, summary_length=150):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="tf", truncation=True, max_length=max_length)
    # inputs = inputs.to("cuda")

    # Generate summary
    summary_tokens = model.generate(
        **inputs,
        max_length=summary_length,
        min_length=50,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    # Decode the summary
    summary_text = tokenizer.decode(summary_tokens[0], skip_special_tokens=True)

    return summary_text

In [25]:
# Translation and Summarization Pipeline
def translation_summarization_pipeline(data,num_samples=5):
    translated_summaries = []

    # Process only the first `num_samples` rows for initial check
    data_sample = data.head(num_samples)

    for i, row in data_sample.iterrows():
        text = row['text']
        lang = row['language']

        # print("text: ",text)
        print(f"Processing row {i} with language: {lang}")

        # Skip empty texts
        if not text or len(text.strip()) == 0:
            print(f"Skipping empty text at row {i}")
            continue

        # Translate if the article is not in English
        if lang != "en":
          print(f"Translating text from {lang} to English")
          translated_text = translate_text(text, source_lang=lang)
          print(translated_text)
        else:
            translated_text = text

        # Summarize the translated text
        summary = fine_tuned_summarize_text(translated_text, summarization_model, summarization_tokenizer)

        # Store result with original metadata
        translated_summaries.append({
            "original_text": text,
            "translated_text": translated_text,
            "summary": summary,
            "language": lang
        })

    return pd.DataFrame(translated_summaries)

In [26]:
# Running the pipeline
processed_data = translation_summarization_pipeline(data,num_samples=5)

# View the processed data
print(processed_data.head())

Processing row 0 with language: de
Translating text from de to English
Transport in cattle carriage, smell of meat in the air, selection with Dr Mengele: Holocaust survivor Lisa Miková remembers the Auschwitz-Birkenau extermination camp. Lisa Miková was born in Prague in 1922. In her largely secular Jewish family, German and Czech were spoken. In 1942, she was deported to the Theresienstadt concentration camp. From there, her parents first became, then, in the autumn of 1944, her husband František was transported to the Auschwitz extermination camp. Lisa Miková volunteered for a women's transport shortly afterwards. Open detailed view Lisa Miková in 1939. At that time, she experienced how the Wehrmacht invaded Prague. The Germans immediately expropriated her father's business. (Photo: oh) What torments Lisa Miková experienced and how she survived, she repeatedly describes to Czech and
Processing row 1 with language: de
Translating text from de to English
Marble zebra strips, pompous bu

**Processing row 0 with language: de**  
Translating text from de to English  
Transport in cattle carriage, smell of meat in the air, selection with Dr Mengele: Holocaust survivor Lisa Miková remembers the Auschwitz-Birkenau extermination camp. Lisa Miková was born in Prague in 1922. In her largely secular Jewish family, German and Czech were spoken. In 1942, she was deported to the Theresienstadt concentration camp. From there, her parents first became, then, in the autumn of 1944, her husband František was transported to the Auschwitz extermination camp. Lisa Miková volunteered for a women's transport shortly afterwards. Open detailed view Lisa Miková in 1939. At that time, she experienced how the Wehrmacht invaded Prague. The Germans immediately expropriated her father's business. (Photo: oh) What torments Lisa Miková experienced and how she survived, she repeatedly describes to Czech and

---

**Processing row 1 with language: de**  
Translating text from de to English  
Marble zebra strips, pompous buildings: Sindelfingen was once considered - thanks to Daimler - to be the richest municipality in Europe. Precisely because of this, the city is now stuck. German cities' coffers are empty - with some bizarre ideas they want to fight the crisis. sueddeutsche.de presents a series of special municipalities and creative savings plans. Open detailed view Protest in Sindelfingen: Many citizens do not want to accept the planned closure of the Klostergartenschule. (Photo: Photo: buergerentscheid-klostergarten.de) 25 instead of 26 degrees! Even the water temperature in Sindelfingen's opulent bathing centre lowered the municipality in order to save money - and to bring the finances of the Daimler city back into balance. The desperate attempt to consolidate the budget three years ago failed miserably: "That went

---

**Processing row 2 with language: de**  
Translating text from de to English  
This Monday, when the country’s leaders of the left are discussing the successors of the current leaders Lothar Bisky and Oskar Lafontaine, as well as the federal executive Dietmar Bartsch, it’s not just about who should lead the party. It’s also about the future orientation and strength of a party that was mainly held together by Lafontaine. Five years ago, he finally succeeded in forming a party from the East German PDS and the West German WASG. However, a party that is divided into East and West, into willing and eternal oppositionists, into realists and ideologists, into moderate and radical leftists. We present possible candidates. Vote: Who do you think is suitable and who is incapable? Fight for Lafontaine’s legacy: Gregor Gysi If anyone should lead the party alone, as many East Germans wish, only the 62

---

**Processing row 3 with language: de**  
Translating text from de to English  
The portrait of 1791 shows Haiti's national hero François-Dominique Toussaint L'Ouverture. He was one of the leaders of the revolution in Haiti and the author of the first constitution. The roots of misery lie in the past. Haiti still pays for its liberation 200 years ago. Even then, the world's major powers did not take the island state seriously. Last weekend, the British architect and founder of the organization Architecture for Humanity sent a breathless, desperate e-mail to his friends and supporters. "Not earthquakes, but buildings kill people," he wrote in the subject line. Thus, he brought to the point what the geologist and author Simon Winchester or the urbanist Mike Davis have repeatedly written - there are no natural disasters. There are only violent natural events, which have fatal consequences. The consequence of this conclusion

---

**Processing row 4 with language: de**  
Translating text from de to English  
New heads and old acquaintances: Angela Merkel's new government team had some surprises in store. Meanwhile, the ministers have found themselves in their new job - some with more, the others with less success. CANCELLER: Angela Merkel (55, CDU) Angela Merkel is now where she wanted to be in 2005: She is the chancellor of a black-yellow coalition. However, she is also one of those in the Union who have learned to appreciate the great coalition with the SPD in the past four years. In the end, it was speculated that she would have preferred to continue to govern with the SPD rather than to start an unpredictable new beginning with the FDP. How unpredictable this new beginning should be, however, probably even the most far-sighted chancellor could not have foreseen. There is no lack of disputes and differences between the

---

**Processing row 5 with language: de**  
Translating text from de to English  
The black-yellow coalition is crunching. Although the coalition agreement was preceded by intensive negotiations, although the parties have now had three months to get together. It is crunching so much that the party leaders of the CDU, CSU and FDP want to meet again in mid-January in order to get the engine running again. Five major conflicts at a glance: Tax cuts The conflict: The citizens are to be relieved of EUR 24 billion per year. This is what the coalition agreement states. The goal is to be achieved by the end of the legislative period at the latest. The controversial "growth acceleration law" - in force since the beginning of the year - is the first step. Others are to follow. But the CSU suddenly does not want to know any more. 24 billion is too much. It still sees a maximum

---

**Processing row 6 with language: de**  
Translating text from de to English  
Are German politicians available for sale? The debate about party donations is in full swing - triggered by the donation of the hotel entrepreneur August Baron von Finck (here with his wife Francine), who had transferred a total of 1.1 million euros to the FDP between 2008 and 2009. And who was able to rejoice after the election that the black-and-yellow government lowered the VAT rate on hotel stays. The Greens are now calling for a limit on party donations. The Left Party wants to ban them altogether. However, both parties see little of the money rain - CDU and FDP, on the other hand, a ban would be expensive. They benefit greatly from the financial contributions of industrialists, associations and individuals. Who are the people who donate hundreds of thousands of euros to politics? An overview in pictures. Photo: dpa

---

**Processing row 7 with language: de**  
Translating text from de to English  
The A400M project is at a standstill - once again. The manufacturer Airbus wants to see money from the buyers, otherwise - according to the threat - the military aircraft will be abandoned. Now, the secretaries of state of the recipient states meet to discuss their further action. Behind all parties lies a long-standing struggle for money, power and technology. The story of a crash aircraft. 1991 In the beginning there is a problem - and that is Transall C-160 (photo). The military transport developed jointly by the French and Germans was already in the late fifties and was accordingly obsolete. In the context of the current Afghanistan operation, 40 years old aircraft fly to the Hindu Kush, time annoys - because the range of the transporters is too small, this can only be done with a stopover. A new aircraft must be produced - and in order to develop it, the European manufacture

---

**Processing row 8 with language: de**  
Translating text from de to English  
The world leader plays without Germany: Only four German companies made it into the top 100 of the most valuable companies. Asian companies are the winners. Ernst & Young auditors have determined the most expensive companies in the world: In the top 100 there are only four German companies. Last year there were eight companies. In other countries, the recovery of the stock markets has been stronger, the study says. The weak performance of the German companies surprises even the editors. The leader of the hit parade and thus the most valuable company in the world is the Chinese oil company Petrochina, whose market capitalization is indicated with about 353 billion dollars. This figure reflects what a company is worth on the stock exchange. Photo: AFP

---

**Processing row 9 with language: de**  
Translating text from de to English  
In helping the victims of the earthquake in Haiti, the initial problems have been overcome, according to UN Secretary-General Ban Ki Moon. "I know there were some delays in the first days. But we have now built a very effective system to avoid bottlenecks," Ban said in New York after a service for the tens of thousands of dead in Haiti. Haitians are waiting for help in the devastated city of Leogane outside aid agencies. "The Haitians are not violent -- they are just hungry. We came here to bring them food," said a rescue worker from the United States. Source: dpa, photo: getty

In [29]:
processed_data.iloc[0]

original_text      Transport im Viehwaggon, Fleischgeruch in der ...
translated_text    Transport in cattle carriage, smell of meat in...
summary            Lisa Miková was born in Prague in 1922. In her...
language                                                          de
Name: 0, dtype: object